In [1]:
pip install flask_cors

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Importing flask module in the project is mandatory
# An object of Flask class is our WSGI application.
from flask import Flask, jsonify, request
from flask_cors import CORS
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import random
import time
import re

import denver
import boulder
import adams
import douglas
import elpaso
import homes
import redfin

# Flask constructor takes the name of 
# current module (__name__) as argument.
app = Flask(__name__)
CORS(app)  # Enable CORS for all routes


# Open the URL
#driver.get("https://www.homes.com/douglas-county-co/")

def search_adams(query):
    return jsonify(adams.scrape_property_info(query))

def search_boulder(query):
    return jsonify(boulder.search_boulder(query))

def search_broomfield(query):
    data_set = {}
    data_set['Broomfield'] = "NULL"
    return(jsonify(data_set))

def search_douglas(query):
    return jsonify(douglas.search_douglas(query))

def search_el_paso(query):
    return(jsonify(elpaso.search_elpaso(query)))

def search_larimer(query):
    data_set = {}
    data_set['Larimer'] = "NULL"
    return(jsonify(data_set))

def search_weld(query):
    data_set = {}
    data_set['Weld'] = "NULL"
    return(jsonify(data_set))

def search_denver(query):
    return jsonify(denver.search_denver(query))

def search_homes(query):
    return jsonify(homes.search_homes(query))

def search_redfin(query):
    return jsonify(redfin.scrape_redfin(query))





@app.route('/search', methods=['POST'])
def search():
    request_data = request.get_json()
    query = request_data.get('search', '').lower()
    method = request_data.get('method', '').lower()

    print(request_data)
    print("County is "+ query)
    print("Method is "+ method)

    search_term = " " + str(query)

    if (method == 'redfin'):
        return (search_redfin(search_term))
    return (search_homes(search_term))
    
    

@app.route('/query', methods=['POST'])
def query():
    # Get the JSON data from the request
    request_data = request.get_json()
    
    # Extract the query from the JSON data
    query = request_data.get('address', '').lower()
    county = request_data.get('county', '').lower()

    print(request_data)
    print("County is " + county)
    print("Query is " + query)

    if (county == 'adams'):
        return(search_adams(query))
    elif (county == 'boulder'):
        return(search_boulder(query))
    elif (county == 'broomfield'):
        return(search_broomfield(query))
    elif (county == 'denver'):
        return(search_denver(query))
    elif (county == 'douglas'):
        return(search_douglas(query))
    elif (county == 'el paso'):
        return(search_el_paso(query))
    elif (county == 'larimer'):
        return(search_larimer(query))
    elif (county == 'weld'):
        return(search_weld(query))
    data_set[county] = "NULL"
    return(jsonify(data_set))





@app.route('/search-property', methods=['GET'])
def search_property():

    number = random.randint(1, 100)  # Generate a random number between 1 and 100
    return jsonify({'number': number})

# The route() function of the Flask class is a decorator, 
# which tells the application which URL should call 
# the associated function.

@app.route('/')
# ‘/’ URL is bound with hello_world() function.
def hello_world():
    return 'Hello World'

@app.route('/random', methods=['GET'])
def random_number():
    number = random.randint(1, 100)  # Generate a random number between 1 and 100
    return jsonify({'number': number})

# main driver function
if __name__ == '__main__':

    # run() method of Flask class runs the application 
    # on the local development server.
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Apr/2025 18:49:03] "OPTIONS /search HTTP/1.1" 200 -


{'search': 'Douglas CO', 'method': 'redfin'}
County is douglas co
Method is redfin


127.0.0.1 - - [07/Apr/2025 18:49:24] "POST /search HTTP/1.1" 200 -


In [17]:
print(returnval)

{'detail0': {'Year': 2020, 'Type': 'property_type', 'Actual': 'actual', 'Assessed': 'assessed', 'Exempt': 'exempt'}, 'detail1': {'Year': 2021, 'Type': 'property_type', 'Actual': 'actual', 'Assessed': 'assessed', 'Exempt': 'exempt'}}
